# NBA Lottery Simulator

Once the season concludes, the NBA lottery odds are determined. However, once the lottery draft order is revealed in reverse, the odds do not get updated. This would be especially interesting when a team is "skipped", meaning that team is guaranteed to pick in the top 4. 

This notebook contains an introduction to the `image` endpoint of the API through the `Logo` class in the `py_ball` package to add some aesthetic appeal to the live-odds calculator.

In [1]:
import glob
from io import BytesIO
import itertools
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import ipywidgets as widgets
import base64
from IPython.display import display, HTML
from IPython.display import clear_output
import time

from py_ball import image

HEADERS = {'Connection': 'close',
           'Host': 'stats.nba.com',
           'Origin': 'http://stats.nba.com',
           'Upgrade-Insecure-Requests': '1',
           'Referer': 'stats.nba.com',
           'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2)' + \
                         'AppleWebKit/537.36 (KHTML, like Gecko) ' + \
                         'Chrome/66.0.3359.117 Safari/537.36'}

pd.options.mode.chained_assignment = None  # Disabling pandas SetWithCopyWarnings

The first step is to write a function that calculates the odds of selecting in any lottery slot given the number of teams that have not been selected, the corresponding number of chances remaining, and the teams that have been selected.

In [2]:
def calculate_pick_probabilities(lotto_combos, top_picks, teams_selected, top_pick_list, top_pick_order):
    """ calculate_pick_probabilities dynamically calculates the probability of each
    team receiving picks 1-14 in the NBA Lottery.

    @param lotto_combos (dict): Dictionary keyed by team
        lottery order with values corresponding to each team's
        lottery chances
    @param top_picks (int): Integer indicating the number of
        picks that are selected via the lottery. The rest of the
        picks are slotted in reverse order of team record
    @param teams_selected (list): List containing the team lottery
        order of teams already revealed in the lottery and not in
        the top_picks number of picks
    @param top_pick_list (list): List containing the team lottery
        order of teams already revealed to be in the top_picks number
        of picks
    @param top_pick_order (list): List containing the order of the
        top picks as they are revealed

    Returns:

        prob_dict (dict): Dictionoary keyed by team lottery order
            with a list containing the probability of the team
            receiving each draft pick
    """

    prob_dict = {}
    total_teams = len(lotto_combos)
    total_combos = 0
    for num in lotto_combos:
        if num not in teams_selected:  # Only adding teams that have not been revealed
            total_combos += lotto_combos[num]

    team_list = list(lotto_combos.keys())
    for team in range(1, total_teams + 1):
        prob_list = [0] * total_teams # Initializing the list of pick probabilities                      
        if team in teams_selected and team not in top_pick_order:
            fall_spot = 0
            for top_pick in top_pick_list:
                if top_pick > team:
                    fall_spot += 1
            prob_list[team - 1 + fall_spot] = 1 # If a team has been revealed, we know with certainty the pick number
        elif team in top_pick_order:
            count = 0
            for team_here in top_pick_order:
                if team_here == team:
                    prob_list[top_picks - count - 1] = 1
                count += 1
        else:
            pick_order = list(itertools.permutations(team_list,
                                                     top_picks - len(top_pick_order)))
            prob_fall = [0] * (top_picks + 1)

            # If a team has not been revealed, we loop through all possible permutations of the
            # first four picks and find the probability of each permutation occurring
            for order in pick_order:
                balls_remaining = total_combos
                other_teams_probability = 1
                top_pick_probability = 1

                # If a team has been revealed to be in the top number of picks, only permutations
                # containing that team are valid. top_pick_ind ensures that only these permutations
                # are evaluated
                top_pick_ind = 1
                for top_pick in top_pick_list:
                    if top_pick in order or top_pick in teams_selected:
                        top_pick_ind *= 1
                    else:
                        top_pick_ind *= 0

                if top_pick_ind:
                    fall_spots = 0
                    pick_ind = 0
                    count = 0
                    for pick in order:
                        if pick in teams_selected:
                            # If a pick has already been revealed to be outside of the
                            # top picks, this permutation is invalid
                            other_teams_probability = 0
                            top_pick_probability = 0
                            break

                        if team in order:
                            # If team is in this permutation of top picks, evaluate the probability
                            # of the permutation occurring AND track the pick number with the pick_count
                            # index.
                            if pick == team:
                                pick_count = count
                            other_teams_probability = 0
                            shot_prob = lotto_combos[pick]/float(balls_remaining)

                            balls_remaining = balls_remaining - lotto_combos[pick]
                            top_pick_probability *= shot_prob
                            pick_ind = 1
                            count += 1
                        else:
                            # If team is not in this permutation, track the number of teams higher
                            # than the current team that are in the permutation. This gives the number
                            # of spots that the team falls in the lottery. Again, track the probability
                            # of this permutation occurring AND track the pick number with the fall_spots
                            # index
                            pick_ind = 0
                            if pick > team:
                                fall_spots += 1

                            shot_prob = lotto_combos[pick]/float(balls_remaining)

                            balls_remaining = balls_remaining - lotto_combos[pick]

                            count += 1

                            other_teams_probability *= shot_prob

                    # If pick_ind == 0, the team does not pick in the top_picks in this permutation
                    # If pick_ind == 1, the team does pick in the top_picks in this permutation
                    if pick_ind == 0:
                        prob_fall[fall_spots] += \
                            other_teams_probability
                    else:
                        prob_list[pick_count] += \
                            top_pick_probability

            # This loop fills in the corresponding "fall" spot with the appropriate probability
            for spot in range(team - 1, team + top_picks - len(top_pick_order)):
                if spot <= total_teams - 1 and spot > top_picks - 1:
                    prob_list[spot] = prob_fall[spot - team + 1]

        prob_list = [x/sum(prob_list) for x in prob_list]
        prob_dict[team] = [round(100*x, 1) for x in prob_list]

    return prob_dict

In [3]:
lottery_info = {1: {'name': 'Knicks', 'id': '1610612752'},
                2: {'name': 'Cavaliers', 'id': '1610612739'},
                3: {'name': 'Suns', 'id': '1610612756'},
                4: {'name': 'Bulls', 'id': '1610612741'},
                5: {'name': 'Hawks', 'id': '1610612737'},
                6: {'name': 'Wizards', 'id': '1610612764'},
                7: {'name': 'Pelicans', 'id': '1610612740'},
                8: {'name': 'Grizzlies', 'id': '1610612763'},
                9: {'name': 'Mavericks', 'id': '1610612742'},
                10: {'name': 'Timberwolves', 'id': '1610612750'},
                11: {'name': 'Lakers', 'id': '1610612747'},
                12: {'name': 'Hornets', 'id': '1610612766'},
                13: {'name': 'Heat', 'id': '1610612748'},
                14: {'name': 'Kings', 'id': '1610612758'}}

for lotto in lottery_info:
    
    team_logo = image.Logo(league='NBA',
                           team_id=lottery_info[lotto]['id'])
    lottery_info[lotto]['logo'] = team_logo.image

## Instructions

1. Tune into the NBA Draft Lottery on Tuesday, May 14th!
2. As picks are revealed, fill in the pick order of the pick that is revealed. For example, if the Kings are revealed first, add the integer 14 to the teams_selected list below.
3. If a pick is "skipped" (an expected team is jumped over in the revealing process), add the integer of that team to the top_pick_list list below.
4. Once the first 10 picks are revealed, the top_pick_list should contain 4 teams. As the final four teams are revealed, add the integer of that team to both the teams_selected_list and top_pick_order list
5. Throughout this entire process, run all cells below in order to show the updated live odds!

### Example

Consider the following draft order (starting with pick 14):
- Kings
- Heat
- Lakers
- Timberwolves
- Mavericks
- Grizzlies
- Pelicans
- Wizards
- Heat
- Bulls
- Knicks
- Suns
- Cavaliers
- Hornets

One would fill in the lists from left to right accordingly
- `teams_selected = [14, 13, 11, 10, 9, 8, 7, 6, 5, 4, 1, 3, 2, 12]`
- `top_pick_list = [12, 3, 2, 1]`
- `top_pick_order = [1, 3, 2, 12]`

In [45]:
lotto_chances = {1: 140, 2: 140, 3: 140,
                 4: 125, 5: 105, 6: 90, 7: 60,
                 8: 60, 9: 60, 10: 30, 11: 20, 12: 10,
                 13: 10, 14: 10}
top_picks = 4
teams_selected = [14, 13, 12, 10, 9, 6, 5, 4, 3, 2, 11, 1, 8, 7]
top_pick_list = [11, 8, 7, 1] # Gets filled in with top picks as they are "revealed"
top_pick_order = [11, 1, 8, 7] # Ordered list of length top_picks, first index is the highest possible pick

prob_dict = calculate_pick_probabilities(lotto_chances, top_picks, teams_selected, top_pick_list, top_pick_order)

In [46]:
lotto_df = pd.DataFrame(prob_dict)

# Coding in the pick conversions that trigger should a certain order be pulled
lotto_df.columns = ['Celtics' if (x == 14 and (prob_dict[x][13]==100 or prob_dict[1]==100 or prob_dict[2]==100 or prob_dict[3]==100)) else
                    '76ers' if (x == 14 and prob_dict[x][0]==100) else
                    'Celtics' if (x == 8 and (prob_dict[x][8]==100 or prob_dict[x][9]==100 or prob_dict[x][10]==100 or prob_dict[x][11]==100)) else
                    'Hawks' if (x == 9 and (prob_dict[x][8]==100 or prob_dict[x][9]==100 or prob_dict[x][10]==100 or prob_dict[x][11]==100 or prob_dict[x][12]==100)) else
                    lottery_info[x]['name'] for x in lottery_info]
lotto_df = lotto_df.T
lotto_df.columns = list(range(1, len(lotto_chances) + 1))
lotto_df = lotto_df.sort_values([1, 2, 3, 4],
                                ascending=[False, False, False, False])

In [47]:
pd.set_option('display.max_colwidth', -1)

def get_thumbnail(i):
    i.thumbnail((150, 150), Image.LANCZOS)
    return i

def image_base64(im):
    if isinstance(im, str):
        im = get_thumbnail(im)
    with BytesIO() as buffer:
        im.save(buffer, 'jpeg')
        return base64.b64encode(buffer.getvalue()).decode()

def image_formatter(im):
    return f'<img src="data:image/jpeg;base64,{image_base64(im)}">'


image_list = []
for lotto in lottery_info:
    image_list.append(get_thumbnail(lottery_info[lotto]['logo']))
lotto_df['logo'] = image_list

In [48]:
HTML(lotto_df.to_html(formatters={'logo': image_formatter}, escape=False))

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,logo
Pelicans,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
Grizzlies,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
Knicks,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
Lakers,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
Cavaliers,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
Suns,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
Bulls,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
Hawks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,
Wizards,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,
Hawks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,


## Conclusion

Be sure to run this notebook live as the draft order is being revealed!